# Monte Carlo simulation study using R

## Goal of the study 1

**Prove that  OLS estimators of Generalised Linear Regression (GLR) model are Best Linear Unbiased Estimators (BLUEs)**

<br>

Generalised Linear Regression mode is given as:

$$
    Y = X\beta + u
$$

where:

* $Y$ is $n\times1$ dependent vector
    
* $X$ is $n\times k$ independent variables matrix
    
* $\beta$ is $k\times1$ unknown parameters vector
    
* $u$ is $n\times1$ error term vector

<br>
Assumptions:

1. $E(u) = 0$ and $E(uu^{\prime}) = \sigma_{u}^{2} I_{u}$ where $\sigma = \sqrt{\frac{\sum_{i = 1}^{n} (x_{i} - \bar{x})^{2}}{n - 1}}$ is standard deviation.

2. $X$ is non-stochastic matrix and $cov(X, u) = 0$

3. $X$ is full column rank matrix, i.e. $rank(X) = k$

<br>
The OLS estimator is given as:

$$
    \hat{\beta}_{OLS} = (X^{\prime}X)^{-1}X^{\prime}Y
$$

<br>
The bias and variance of OLS estimators are given as:

$$bias(\hat{\beta}_{OLS}) = \hat{\beta}_{OLS} - \beta$$

$$var(\hat{\beta}_{OLS}) = \sigma_{u}^{2}(X^{\prime}X)^{-1}$$

In [1]:
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt

import cufflinks as cf
import plotly.offline as opy
import plotly.graph_objs as go

import rpy2

In [2]:
%load_ext rpy2.ipython

In [5]:
%%R

library(ggplot2)

n_trials = 1000
beta = c(1, 3) # true beta
sample_size = 100
X = cbind(1, runif(sample_size, -1, 1))
bias_results = c()
var_results = c()

get_ols_estimator <- function(Y, X) {
    solve(t(X) %*% X) %*% t(X) %*% Y
}

for (trial in 1:n_trials) {
    u = rnorm(sample_size, 0, 1) # error term
    y = X %*% beta + u
    beta_ols = get_ols_estimator(y, X)
    bias_results = c(bias_results, beta_ols - beta)
    var_results = c(var_results, var(u) * solve(t(X) %*% X))
}

results = cbind(mean(bias_results), mean(var_results))
results = as.data.frame(results)
colnames(results) = c('bias', 'variance')
results

         bias    variance
1 1.00325e-05 0.009989517


## Goal of study 2

**Compare the performance of OLS and Generalised Least Squares (GLS) estimators in multiple linear regression model when the errors are correlated with first-order autoregressive, i.e. AR(1)**


The second and third assumptions are still valid but the first assumption is replaced with the following:

$$
    u_t = \rho u_{t-1} + \epsilon_t \quad \forall \quad \epsilon_{t} ~ N(0, \sigma_{\epsilon}^{2})
$$

$$
    E(\epsilon_{t}, \epsilon_{s}) = 0 \quad \forall \quad t \neq s
$$

$$
    E(\epsilon_{t}, u_{t-1}) = 0
$$

<br>
The GLS estimator is given as:

$$
    \hat{\beta}_{GLS} = (X^{\prime}\Omega^{-1}X)^{-1}X^{\prime}\Omega^{-1}Y
$$

<br>
where:

$$
    \Omega = E(uu^{\prime}) = \frac{ \sigma_{\epsilon}^{2} }{ 1 - \rho } \begin{bmatrix}
    1 & \rho & \rho^{2} & \dots  & \rho^{n-1} \\
    \rho & 1 & \rho & \dots  & \rho^{n-2} \\
    \vdots & \vdots & \vdots & \ddots & \vdots \\
    \rho^{n-1} & \rho^{n-2} & \rho^{n-3} & \dots  & 1
\end{bmatrix}
$$

Since the elements of $\Omega$ are usually unknown, $\rho$ can be estimated using:

$$
    \hat{\rho} = \frac{ \sum_{i=2}^{n} \hat{u}_{i}\hat{u}_{i-1} }{ \sum_{i=2}^{n} \hat{u}_{i-1}^{2} }
$$

$$
    \hat{\sigma}_{\epsilon}^{2} = \frac{ \sum_{i=1}^{n} \hat{\epsilon}_{i}^{2} }{ n - K }
$$

<br>
where:

* $\hat{\epsilon}_{1} = \hat{u}\sqrt{1 - \hat{\rho}^{2}}$
* $\hat{\epsilon}_{i} = \hat{u}_{i} - \hat{\rho}\hat{u}_{i - 1} \quad \forall \quad i = 2, ..., N$

<br>
The bias and variance of OLS and GLS estimators in this model are given as:

$$bias(\hat{\beta}_{OLS}) = \hat{\beta}_{OLS} - \beta$$

$$bias(\hat{\beta}_{GLS}) = \hat{\beta}_{GLS} - \beta$$

$$var(\hat{\beta}_{OLS}) = (X^{\prime}X)^{-1}X^{\prime}\Omega X(X^{\prime}X)^{-1}$$

$$var(\hat{\beta}_{GLS}) = (X^{\prime}\Omega^{-1}X)^{-1}$$

In [7]:
%%R

n_trials = 1000
beta = c(1, 1)
sample_size = 100
rho = 0.5
X = cbind(1, runif(sample_size, -1, 1))

estimate_omega <- function(u) {
    n = length(u)
    
    rho_hat = (t(u[-n]) %*% u[-1]) / sum(u[-1]^2)
    dim(rho_hat) = NULL
    
    if (rho_hat > 1) {
        rho_hat = 0.99
    } else if (rho_hat < 0) {
        rho_hat = 0.005
        
    }
    
    epsilon_hat = NA
    epsilon_hat[1] = u[1] * sqrt((1 - rho_hat^2))
    epsilon_hat[2:n] = u[-1] + rho_hat * u[-n]
    sigma2_epsilon_hat = sum(epsilon_hat^2) / (n - 2)
    dim(sigma2_epsilon_hat) = NULL

    v = matrix(NA, nrow=n, ncol=n)
    
    for (i in 1:n) {
        for (j in 1:n) {
            v[i, j] = rho_hat^abs(i - j)
        }
    }   
    
     (sigma2_epsilon_hat / (1 - rho_hat^2)) * v
}

get_gls_estimator <- function(Y, X, omega) {
    solve(t(X) %*% solve(omega) %*% X) %*% t(X) %*% solve(omega) %*% Y
}

total_result = matrix(0, nrow=2, ncol=4)

for (trial in 1:n_trials) {
    epsilon = rnorm(sample_size, 0, 1)
    u = c(0)
    u[1] = epsilon[1] / sqrt(1 - rho^2)
    
    for (i in 2:sample_size) {
        u[i] = rho * u[i - 1] + epsilon[i]
    }
    
    y = X %*% beta + u
    
    omega = estimate_omega(u)
    beta_ols = get_ols_estimator(y, X)
    beta_gls = get_gls_estimator(y, X, omega)
    
    bias_beta_hat_ols = beta_ols - beta
    bias_beta_hat_gls = beta_gls - beta

    var_beta_hat_ols = diag(solve(t(X) %*% X) %*% t(X) %*% omega %*% X %*% solve(t(X) %*% X))
    var_beta_hat_gls = diag(solve(t(X) %*% solve(omega) %*% X))
    
    result = cbind(bias_beta_hat_ols, bias_beta_hat_gls, var_beta_hat_ols, var_beta_hat_gls)
    rownames(result) = c('beta_0', 'beta_1')
    colnames(result) = c('OLS bias', 'GLS bias', 'OLS variance', 'GLS variance')
    
    total_result = total_result + result
}

as.data.frame(total_result / n_trials)

            OLS bias      GLS bias OLS variance GLS variance
beta_0 -0.0001166151  0.0010654905    0.1118213   0.11083473
beta_1  0.0051626033 -0.0006069895    0.1112929   0.06206032
